In [ ]:
#########################################################################
###########------- Deep Learning Inmersion ------------##################
#########################################################################
# Capacitador: André Omar Chávez Panduro
# email: andre.chavez@urp.edu.pe
# Tema : Redes Neuronales Convolucionales - CNN
# version: 2.0
#########################################################################

In [2]:
# Nos vinculamos a nuestro drive, para poder usar facilmente GColab
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


---
## **PARTE I - CONSTRUCCION DE LA RED NEURONAL CONVOLUCIONAL **
---

In [3]:
# Parte 1 - Construir el modelo de CNN

# Importar las librerías y paquetes
from keras.models import Sequential     # Creamos una red paso a paso.
from keras.layers import Conv2D         # Creamos capas de convolucion 2D.
from keras.layers import MaxPooling2D   # Creamos capas de reduccion.
from keras.layers import Flatten        # Aplanado u ordenamiento de los datos.
from keras.layers import Dense          # Creamos la conexion entre las capas de redes totalmente conectadas. 

In [5]:
# Inicializar la CNN / Constructor Sequential / Inicializacion de la Red Neuronal
classifier = Sequential()

In [6]:
# Paso 1 - Convolución
# Añadimos una capa de convolucion
# filters : Numeros de mapas de caracteristicas / Encontrar rasgos interesantes / Filtros .
# kernel_size : Tamaño del filtro.
# input_shape : Tamaño de las imagenes, tamaño de fila, columna y canales de color.
# Por defecto es 128*128 y el canal.
# activation : Funcion de activacion, no linealidad necesaria.
classifier.add(Conv2D(filters = 32,kernel_size = (3, 3),
                      input_shape = (64, 64, 3), 
                      activation = "relu")) # No Linealidad

In [7]:
# Paso 2 - Max Pooling
# Capa de reduccion de características, dimensiones y demás.
classifier.add(MaxPooling2D(pool_size = (2,2)))

In [8]:
# Paso 3 - Flattening
classifier.add(Flatten())

In [9]:
# Paso 4 - Full Connection
# Ni demasiado pequeño ni grande, porque puede ser computacionalmente costoso.
classifier.add(Dense(units = 128, activation = "relu")) # Comenzar con 56 -> oo
classifier.add(Dense(units = 1, activation = "sigmoid")) # Capa de salida

In [10]:
# Paso 4 - Compilar la CNN
classifier.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])

In [11]:
classifier.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 31, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 30752)             0         
_________________________________________________________________
dense (Dense)                (None, 128)               3936384   
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 3,937,409
Trainable params: 3,937,409
Non-trainable params: 0
_________________________________________________________________


---
## **PARTE II - AJUSTE DE LA RED NEURONAL CONVOLUCIONAL **
---

In [12]:
# Parte 2 - Ajustar la CNN a las imágenes para entrenar 
from keras.preprocessing.image import ImageDataGenerator

In [13]:
# Data de entrenamiento!
train_datagen = ImageDataGenerator(
        rescale=1./255, # Transforma los pixeles , valores de 0 a 1. La red aprende mas rapido.
        shear_range=0.2,# Direccion de rotacion contra el reloj
        zoom_range=0.2, # Podemos jugar con los parametros pero agrega mucho costo computacional / Zoom
        horizontal_flip=True) # Va poner la imagen horizontal aleatoriamente

In [14]:
# Data de test!
test_datagen = ImageDataGenerator(rescale=1./255)

In [15]:
# Leemos los datos de entrenamiento!
training_dataset = train_datagen.flow_from_directory('/content/drive/MyDrive/DeepLearningInmersionG3/Clase02/LaboratorioAplicacionN°02/dataset/training_set',
                                                    target_size=(64, 64),
                                                    batch_size=32, # Tamaño del bloque antes de actualizar los pesos.
                                                    class_mode='binary') # En modo de class se especifica el numero de categorias

Found 7999 images belonging to 2 classes.


In [16]:
# Leemos los datos de test!
testing_dataset = test_datagen.flow_from_directory('/content/drive/MyDrive/DeepLearningInmersionG3/Clase02/LaboratorioAplicacionN°02/dataset/test_set',
                                                target_size=(64, 64), # Tamaño de carga
                                                batch_size=32,        # Tamaño de los lotes
                                                class_mode='binary')

Found 1999 images belonging to 2 classes.


In [17]:
# Entrenamiento de la red neuronal convolucional!
classifier.fit_generator(training_dataset,
                         steps_per_epoch=8000,
                        epochs=5,          # Este valor debe ser entre 15 a 25.
                        validation_data=testing_dataset,
                        validation_steps=2000)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5
8000/8000 [==============================] - 2241s 278ms/step - loss: 0.7099 - accuracy: 0.5684 - val_loss: 0.6273 - val_accuracy: 0.6488


---
## **PARTE III - PREDICCION DE NUEVOS REGISTROS **
---

In [ ]:
# Parte 3 - Cómo hacer nuevas predicciones
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = classifier.predict(test_image)
training_dataset.class_indices
if result[0][0] == 1:
    prediction = 'dog'
else:
    prediction = 'cat'


In [ ]:
# Fin!